# PCA - Tarefa 01: *HAR* com PCA

Vamos trabalhar com a base da demonstração feita em aula, mas vamos explorar um pouco melhor como é o desempenho da árvore variando o número de componentes principais.

In [1]:
import pandas as pd
import os
import numpy as np
import time

from sklearn.tree import DecisionTreeClassifier

from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split

In [2]:
folder = r"Dados\UCI"

features_file = os.path.join(folder, "features.txt")
labels_file = os.path.join(folder, "activity_labels.txt")

subjtrain_file = os.path.join(folder, "subject_train.txt")
xtrain_file = os.path.join(folder, "X_train.txt")
ytrain_file = os.path.join(folder, "y_train.txt")

subjtest_file = os.path.join(folder, "subject_test.txt")
xtest_file = os.path.join(folder, "X_test.txt")
ytest_file = os.path.join(folder, "y_test.txt")

# Carregando a base features.txt em uma Series
features = pd.read_csv(features_file, header=None, names=['nome_var'], sep="#")
labels = pd.read_csv(labels_file, delim_whitespace=True, header=None, names=['cod_label', 'label'])

subject_train = pd.read_csv(subjtrain_file, header=None, names=['subject_id'])
X_train = pd.read_csv(xtrain_file, delim_whitespace=True, header=None, names=features['nome_var'].tolist())
y_train = pd.read_csv(ytrain_file, header=None, names=['cod_label'])

subject_test = pd.read_csv(subjtest_file, header=None, names=['subject_id'])
X_test = pd.read_csv(xtest_file, delim_whitespace=True, header=None, names=features['nome_var'].tolist())
y_test = pd.read_csv(ytest_file, header=None, names=['cod_label'])

## Árvore de decisão

Rode uma árvore de decisão com todas as variáveis, utilizando o ```ccp_alpha=0.001```. Avalie a acurácia nas bases de treinamento e teste. Avalie o tempo de processamento.

In [3]:
%%time
# Arvore de decisão com ccp_alpha=0.001
tree = DecisionTreeClassifier(ccp_alpha=0.001)
tree.fit(X_train, y_train)

# Previsões do modelo
y_test_pred = tree.predict(X_test)
y_train_pred = tree.predict(X_train)

# Acurácia do modelo no conjunto de treinamento e teste
acuracia_train = accuracy_score(y_train, y_train_pred)
acuracia_test = accuracy_score(y_test, y_test_pred)

# Print das acuracias
print(f'Acuracia treinamento: {acuracia_train*100:.2f}')
print(f'Acuracia teste: {acuracia_test*100:.2f}')

Acuracia treinamento: 97.58
Acuracia teste: 87.92
CPU times: total: 10.2 s
Wall time: 11.2 s


## Árvore com PCA

Faça uma análise de componentes principais das variáveis originais. Utilize apenas uma componente. Faça uma árvore de decisão com esta componente como variável explicativa.

- Avalie a acurácia nas bases de treinamento e teste
- Avalie o tempo de processamento

In [6]:
# Criando as bases de treino, teste e validação
X_train_pca, X_valida_pca, y_train_pca, y_valida_pca = train_test_split(X_train, y_train)

In [7]:
%%time

# Inicialmente crio o modelo PCA para poder posteriormente avaliar a acurácia com apenas uma componente
prcomp = PCA().fit(X_train_pca)

pc_treino = prcomp.transform(X_train_pca)
pc_valida = prcomp.transform(X_valida_pca)
pc_teste  = prcomp.transform(X_test)

pc_treino.shape

CPU times: total: 1.25 s
Wall time: 603 ms


(5514, 561)

In [8]:
%%time
# Escolhendo a componente
n=1

colunas = ['cp'+str(x+1) for x in list(range(n))]

pc_train = pd.DataFrame(pc_treino[:,:n], columns = colunas)
pc_valida = pd.DataFrame(pc_valida [:,:n], columns = colunas)
pc_test  = pd.DataFrame( pc_teste[:,:n], columns = colunas)

# Criação da arvore e treinamento. Após treinamento, realiza o post-prunning
clf = DecisionTreeClassifier(random_state=42).fit(pc_train, y_train_pca)

caminho = DecisionTreeClassifier(random_state=42, min_samples_leaf=20).cost_complexity_pruning_path(pc_train, y_train_pca)
ccp_alphas, impurities = caminho.ccp_alphas, caminho.impurities

ccp_alphas = np.unique(ccp_alphas[ccp_alphas>=0])

clfs = []
for ccp_alpha in ccp_alphas:
    clf = DecisionTreeClassifier(random_state=42, ccp_alpha=ccp_alpha).fit(pc_train, y_train_pca)
    clfs.append(clf)

# Calculando as acuracias e printando
train_scores = [clf.score(pc_train, y_train_pca) for clf in clfs]
valid_scores = [clf.score(pc_valida, y_valida_pca) for clf in clfs]

ind_melhor_arvore = len(valid_scores) - valid_scores[::-1].index(max(valid_scores)) - 1
melhor_arvore = clfs[ind_melhor_arvore]

print(f'Acuracia PCA Treinamento: {train_scores[ind_melhor_arvore]*100:.2f}')
print(f'Acuracia PCA Validação:   {valid_scores[ind_melhor_arvore]*100:.2f}')
print(f'Acuracia PCA Teste:       {melhor_arvore.score(pc_test, y_test)*100:.2f}')

Acuracia PCA Treinamento: 49.96
Acuracia PCA Validação:   49.02
Acuracia PCA Teste:       45.47
CPU times: total: 7.22 s
Wall time: 7.83 s


## Testando o número de componentes

Com base no código acima, teste a árvore de classificação com pelo menos as seguintes possibilidades de quantidades de componentes: ```[1, 2, 5, 10, 50]```. Avalie para cada uma delas:

- Acurácia nas bases de treino e teste
- Tempo de processamento


In [10]:
valores_de_n = [2, 5, 10, 50]

In [11]:
for n in valores_de_n:
    colunas = ['cp' + str(x + 1) for x in range(n)]

    pc_train = pd.DataFrame(pc_treino[:, :n], columns=colunas)
    pc_valida = pd.DataFrame(pc_valida.iloc[:, :n], columns=colunas)
    pc_test = pd.DataFrame(pc_teste[:, :n], columns=colunas)

    # Inclusão da contagem de tempo para o loop
    start_time = time.time()

    clf = DecisionTreeClassifier(random_state=42).fit(pc_train, y_train_pca)

    caminho = DecisionTreeClassifier(random_state=42, min_samples_leaf=20).cost_complexity_pruning_path(pc_train,
                                                                                                           y_train_pca)
    ccp_alphas, impurities = caminho.ccp_alphas, caminho.impurities

    ccp_alphas = np.unique(ccp_alphas[ccp_alphas >= 0])

    clfs = []
    for ccp_alpha in ccp_alphas:
        clf = DecisionTreeClassifier(random_state=42, ccp_alpha=ccp_alpha).fit(pc_train, y_train_pca)
        clfs.append(clf)

    # Termino da contagem de tempo
    end_time = time.time()

    # Calculando as acuracias
    train_scores = [clf.score(pc_train, y_train_pca) for clf in clfs]
    valid_scores = [clf.score(pc_valida, y_valida_pca) for clf in clfs]

    # Encontrando o índice da melhor árvore durante a validação
    ind_melhor_arvore = np.argmax(valid_scores)

    # Se o ind_melhor_arvore for 0, então estamos usando o modelo sem poda
    if ind_melhor_arvore == 0:
        melhor_arvore = clfs[0]
    else:
        melhor_arvore = clfs[ind_melhor_arvore - 1]
    
    print(f'Tempo de execução para n={n}: {end_time - start_time:.2f} segundos')
    print(f'Acuracia PCA Treinamento para n={n}: {train_scores[ind_melhor_arvore] * 100:.2f}')
    print(f'Acuracia PCA Validação para n={n}:   {valid_scores[ind_melhor_arvore] * 100:.2f}')
    print(f'Acuracia PCA Teste para n={n}:       {melhor_arvore.score(pc_test, y_test) * 100:.2f}')
    print('-' * 40)

Tempo de execução para n=2: 8.82 segundos
Acuracia PCA Treinamento para n=2: 59.10
Acuracia PCA Validação para n=2:   46.90
Acuracia PCA Teste para n=2:       60.06
----------------------------------------
Tempo de execução para n=5: 8.73 segundos
Acuracia PCA Treinamento para n=5: 90.32
Acuracia PCA Validação para n=5:   38.36
Acuracia PCA Teste para n=5:       76.93
----------------------------------------
Tempo de execução para n=10: 16.07 segundos
Acuracia PCA Treinamento para n=10: 69.68
Acuracia PCA Validação para n=10:   35.53
Acuracia PCA Teste para n=10:       77.40
----------------------------------------
Tempo de execução para n=50: 76.11 segundos
Acuracia PCA Treinamento para n=50: 69.68
Acuracia PCA Validação para n=50:   35.53
Acuracia PCA Teste para n=50:       75.47
----------------------------------------


## Conclua

- O que aconteceu com a acurácia?
- O que aconteceu com o tempo de processamento?